In [1]:
import pickle
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torchvision import transforms
import numpy as np
from einops import rearrange
import torch
from torchvision.models.video.resnet import VideoResNet, BasicBlock, R2Plus1dStem, Conv2Plus1D
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import gc

USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print('학습을 진행하는 기기:', device)

save_model_file_path = './save_model/{}_{}.{}'

True
학습을 진행하는 기기: cuda:0


In [2]:
with open('./processed_data/train/training_set.dat', "rb") as training_file:
    x1 = pickle.load(training_file)
with open('./processed_data/train/training_set(ElasticTransformation).dat', "rb") as training_file:
    x2 = pickle.load(training_file)
with open('./processed_data/train/training_set(GausianBlur).dat', "rb") as training_file:
    x3 = pickle.load(training_file)
with open('./processed_data/train/training_set(Salt).dat', "rb") as training_file:
    x4 = pickle.load(training_file)

In [3]:
x = x1+ x2+ x3+ x4
del x1, x2, x3, x4
gc.collect()

0

In [4]:
with open('./processed_data/test/test_set.dat', "rb") as training_file:
    y = pickle.load(training_file)

In [5]:
def one_hot_encode(labels, num_classes):
    one_hot_labels = np.zeros((len(labels), num_classes))
    for i in range(len(labels)):
        one_hot_labels[i, labels[i]] = 1
    return one_hot_labels

In [6]:
temp_x = one_hot_encode(x[1], 10)
temp_y = one_hot_encode(y[1], 10)

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.hub import load_state_dict_from_url
import torchvision
from functools import partial
from collections import OrderedDict
import math

import os,inspect,sys

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.insert(0,currentdir)

def convert_relu_to_swish(model):
        for child_name, child in model.named_children():
            if isinstance(child, nn.ReLU):
                setattr(model, child_name, nn.SiLU(True))
                # setattr(model, child_name, Swish())
            else:
                convert_relu_to_swish(child)

class Swish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.mul_(torch.sigmoid(x))

class r2plus1d_18(nn.Module):
    def __init__(self, pretrained=False, num_classes=10, dropout_p=0.5):
        super(r2plus1d_18, self).__init__()
        self.pretrained = pretrained
        self.num_classes = num_classes
        model = torchvision.models.video.r2plus1d_18(pretrained=self.pretrained)
        # delete the last fc layer
        modules = list(model.children())[:-1]
        # print(modules)
        self.r2plus1d_18 = nn.Sequential(*modules)
        convert_relu_to_swish(self.r2plus1d_18)
        self.fc1 = nn.Linear(model.fc.in_features, self.num_classes)
        self.dropout = nn.Dropout(dropout_p, inplace=True)
        # self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        out = self.r2plus1d_18(x)
        # print(out.shape)
        # Flatten the layer to fc
        out = out.flatten(1)
        out = self.dropout(out)
        out = self.fc1(out)
        # out = self.softmax(out)
        return out

class flow_r2plus1d_18(nn.Module):
    def __init__(self, pretrained=False, num_classes=10, dropout_p=0.5):
        super(flow_r2plus1d_18, self).__init__()
        self.pretrained = pretrained
        self.num_classes = num_classes
        model = torchvision.models.video.r2plus1d_18(pretrained=self.pretrained)

        model.stem[0] = nn.Conv3d(2, 45, kernel_size=(1, 7, 7),
                            stride=(1, 2, 2), padding=(0, 3, 3),
                            bias=False)

        # delete the last fc layer
        modules = list(model.children())[:-1]
        # print(modules)
        self.r2plus1d_18 = nn.Sequential(*modules)
        convert_relu_to_swish(self.r2plus1d_18)
        self.fc1 = nn.Linear(model.fc.in_features, self.num_classes)
        self.dropout = nn.Dropout(dropout_p, inplace=True)
    def forward(self, x):
        # print(x.size())
        out = self.r2plus1d_18(x)
        # print(out.shape)
        # Flatten the layer to fc
        out = out.flatten(1)
        out = self.dropout(out)
        out = self.fc1(out)

        return out

In [8]:
model = r2plus1d_18(num_classes = 10).to(device)

/home/ssrlab/ENTER/envs/cuda_test/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ssrlab/ENTER/envs/cuda_test/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
# from torchinfo import summary

# summary(model, input_size = (4,3,15,224,224), col_names = ['input_size','output_size','num_params'], verbose=1)

In [10]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [11]:
# train_transform = A.Compose([
#     A.GaussNoise(always_apply=False, p = 0.3, var_limit = (50.00, 100.00), per_channel = True, mean = 0.0),
#     A.RGBShift(always_apply=False, p = 0.3, r_shift_limit=(-10,10), g_shift_limit=(-10,10), b_shift_limit=(-10,10))])

# test_transform = A.Compose([
    
# ])

In [12]:
class SignLanGuageDataset(Dataset):
    def __init__(self,imagedata,tagdata):
        self.imagedata=imagedata
        self.tagdata=tagdata

    def __len__(self):
        return len(self.imagedata)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image_data=(self.imagedata[idx])
        image_data=torch.FloatTensor(image_data)
        label=self.tagdata[idx]
        label = torch.FloatTensor(label)
        return image_data,label

In [13]:
batchsz = 4
num_workerssz = 4
epochs = 120

In [14]:
train_dataset = SignLanGuageDataset(imagedata=x[0],tagdata=temp_x)
valid_dataset = SignLanGuageDataset(imagedata=y[0],tagdata=temp_y)

In [15]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batchsz, shuffle=False, num_workers=num_workerssz)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=batchsz, shuffle=False, num_workers=num_workerssz)

In [16]:
with torch.cuda.device(0):
    
    for epoch in range(60):
        train_avg_loss = 0
        val_avg_loss = 0
        
        model.train()
        for data, target in tqdm(train_dataloader):
            data = rearrange(data, 'b d h w c -> b c d h w')
            data = data.to(device)
            
            target = target.to(device)
            optimizer.zero_grad()
            output = model(data)

            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_avg_loss += loss
        train_avg_loss = train_avg_loss/len(train_dataloader)
        print('Epoch = {}, train_loss = {}'.format(epoch+1, train_avg_loss))
        with torch.no_grad(): # valid
            model.eval()
            for data, target in tqdm(valid_dataloader):
                data = rearrange(data, 'b d h w c -> b c d h w')
                data = data.to(device)
                
                target = target.to(device)
                
                hypothesis = model(data)
                val_loss = criterion(hypothesis, target)
                val_avg_loss += val_loss
                
            val_avg_loss = val_avg_loss/len(valid_dataloader)

        torch.save({
            'epoch': epoch+1,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, save_model_file_path.format('model',(epoch+1),'pth'))    
        
        print('Epoch = {}, val_loss = {}'.format(epoch+1, val_avg_loss))
        print('\n')

100%|██████████| 15/15 [00:06<00:00,  2.37it/s]


Epoch = 1, train_loss = 2.432539701461792


100%|██████████| 5/5 [00:00<00:00,  8.66it/s]


Epoch = 1, val_loss = 6.186643123626709




100%|██████████| 15/15 [00:05<00:00,  2.51it/s]


...

Epoch = 59, val_loss = 1.3906329870224




100%|██████████| 15/15 [00:05<00:00,  2.50it/s]


Epoch = 60, train_loss = 0.004456115886569023


100%|██████████| 5/5 [00:00<00:00,  8.57it/s]


Epoch = 60, val_loss = 1.4799762964248657




In [ ]:
# checkpoint=torch.load("./save_model/model_33.pth", map_location=device)
# model.load_state_dict(checkpoint["model"])
# optimizer.load_state_dict(checkpoint["optimizer"])
# start_epoch = checkpoint['epoch']

In [ ]:
with torch.no_grad():
    model.eval()
    for data, target in tqdm(valid_dataloader):
        data = rearrange(data, 'b d h w c -> b c d h w')
        data = data.to(device)
        
        target = target.to(device)
        
        hypothesis = model(data)
        
        # print('target = ', target,'\n', 'hypothesis = ', hypothesis, '\n\n\n\n')
        for i in range(batchsz):
            print('target = ', torch.argmax(target[i]),'\n', 'hypothesis = ', torch.argmax(hypothesis[i]), '\n\n\n\n')           

 20%|██        | 1/5 [00:00<00:00,  4.77it/s]

target =  tensor(9, device='cuda:0') 
 hypothesis =  tensor(9, device='cuda:0') 




target =  tensor(4, device='cuda:0') 
 hypothesis =  tensor(4, device='cuda:0') 




target =  tensor(0, device='cuda:0') 
 hypothesis =  tensor(6, device='cuda:0') 




target =  tensor(7, device='cuda:0') 
 hypothesis =  tensor(7, device='cuda:0') 




target =  

 60%|██████    | 3/5 [00:00<00:00,  7.19it/s]

tensor(5, device='cuda:0') 
 hypothesis =  tensor(5, device='cuda:0') 




target =  tensor(9, device='cuda:0') 
 hypothesis =  tensor(9, device='cuda:0') 




target =  tensor(6, device='cuda:0') 
 hypothesis =  tensor(6, device='cuda:0') 




target =  tensor(8, device='cuda:0') 
 hypothesis =  tensor(9, device='cuda:0') 




target =  tensor(4, device='cuda:0') 
 hypothesis =  tensor(2, device='cuda:0') 




target =  tensor(3, device='cuda:0') 
 hypothesis =  tensor(3, device='cuda:0') 




target =  tensor(7, device='cuda:0') 
 hypothesis =  tensor(6, device='cuda:0') 




target =  tensor(8, device='cuda:0') 
 hypothesis =  tensor(9, device='cuda:0') 




target =  

100%|██████████| 5/5 [00:00<00:00,  6.91it/s]

tensor(2, device='cuda:0') 
 hypothesis =  tensor(2, device='cuda:0') 




target =  tensor(1, device='cuda:0') 
 hypothesis =  tensor(1, device='cuda:0') 




target =  tensor(3, device='cuda:0') 
 hypothesis =  tensor(3, device='cuda:0') 




target =  tensor(6, device='cuda:0') 
 hypothesis =  tensor(6, device='cuda:0') 




target =  tensor(1, device='cuda:0') 
 hypothesis =  tensor(1, device='cuda:0') 




target =  tensor(0, device='cuda:0') 
 hypothesis =  tensor(0, device='cuda:0') 




target =  tensor(2, device='cuda:0') 
 hypothesis =  tensor(2, device='cuda:0') 




target =  tensor(5, device='cuda:0') 
 hypothesis =  tensor(5, device='cuda:0') 






In [ ]:
# len(valid_dataset)

In [ ]:
# valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=20, shuffle=False, num_workers=num_workerssz)

# with torch.no_grad(): # valid
#             model.eval()
#             for data, target in tqdm(valid_dataloader):
#                 data = rearrange(data, 'b d h w c -> b c d h w')
#                 data = data.to(device)
                
#                 target = target.to(device)
                
#                 hypothesis = model(data)

In [ ]:
hypothesis.shape

torch.Size([4, 10])

## 사용 가능한 loss 함수

In [ ]:
# class LabelSmoothingCrossEntropy(nn.Module):
#     def __init__(self):
#         super(LabelSmoothingCrossEntropy, self).__init__()
#     def forward(self, x, target, smoothing=0.1):
#         confidence = 1. - smoothing
#         logprobs = F.log_softmax(x, dim=-1)
#         nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
#         nll_loss = nll_loss.squeeze(1)
#         smooth_loss = -logprobs.mean(dim=-1)
#         loss = confidence * nll_loss + smoothing * smooth_loss
#         return loss.mean()